## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных:
- clients - название таблицы
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

Общий план:
- определить и заполнить пропущенные значения
- замените вещественный тип данных на целочисленный
- удалить дубликаты
- выделить леммы в значениях столбца с целями получения кредита
- категоризировать данные
 
Вопросы, на которые требуется найти ответ:
   - Есть ли зависимость между наличием детей и возвратом кредита в срок?
   - Есть ли зависимость между семейным положением и возвратом кредита в срок?
   - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
   - Как разные цели кредита влияют на его возврат в срок?
   - Какой общий вывод о зависимости раздичных факторов и сроках возврата кредита?

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
clients = pd.read_csv('/datasets/data.csv')


In [2]:
clients.head(40)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
clients.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [5]:
clients.children.value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [6]:
clients.education.value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

## Вывод

 Выводим первые 40 записей (меньше - малоинформативно), чтобы познакомится с таблицей - какие столбцы, какой тип записи данных. Выводим информацию о таблице.
Функцией describe смотрим показатели по стобцам с численными значениями.
Обнаружили некорректные данные:
- есть записи в days_employed с отрицательным значением (это ошибка, для исправления применим функцию модуля числа)
- для days_employed и total_income имеют значение non-null меньшее, чем для остальных столбцов и равное количеству записей, а также из вывода видно наличие NaN в одних в записях с одним номером. Заменим пропуски на медиану после исправления ошибки с отрицательными значениями для days_employed, для total_income можно использовать среднее, т.к. числа расположены в небольшом диапазоне.
- для столбца children есть значение -1 и 20 (явные ошибки - заменим на среднее значение, т.к. число детей варьирует незначительно). Дополнительно смотрим количество по ошибкам. Если бы количество некорректных значений было пару штук, можно было бы просто удалить ячейки, тут 123, поэтому преобразуем.
- в строке education используется разный регистр, но при подстчете уникальних значений все распознается корректно - нечего можно не менять


### Шаг 2. Предобработка данных

### Обработка пропусков

In [7]:
clients['days_employed'] = abs(clients['days_employed']) # избавляемся от отрицательных значений для количества дней

In [8]:
days_employed_med = clients['days_employed'].median()  # считаем медианное значение для days_employed
clients['days_employed'] = clients['days_employed'].fillna(days_employed_med) # заменяем пустые значения на медианные

In [9]:
total_income_mean = clients['total_income'].mean() # считаем среднее значение для total_income
clients['total_income'] = clients['total_income'].fillna(total_income_mean) # заменяем пустые значения на средние

In [10]:
children_mean = int(clients['children'].mean()) # считаем среднее значение числа детей (целочисленное) и заменяем на него ошибочные
clients['children'] = clients['children'].replace(-1, children_mean)
clients['children'] = clients['children'].replace(20, children_mean)

In [11]:
clients.head(30)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [12]:
clients.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.470476,60378.032733,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,0.750534,133257.558514,12.574584,0.548138,1.420324,0.272661,9.763293e+04
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1025.608174,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,2194.220567,42.000000,1.000000,0.000000,0.000000,1.564003e+05
75%,1.000000,4779.587738,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод


В days_employed и total_income хранятся количественные переменные.
Пропуски вероятно, полностью случайные, т.к. параметры не зависят ни от каких других значений. Но при этом эти пункты должны были точно быть внесены в анкету, т.к. критичны пр принятии решения о выдаче кредита. Возможно, связаны с обработкой данных таблицы. Заменили пропуски на медиану после исправления ошибки с отрицательными значениями для days_employed, для total_income можно использовать среднее, т.к. числа расположены в небольшом диапазоне.
Также избавались от некорректных значений для кличества детей.
Оценили обновленный вмд таблицы.

In [13]:
clients.children.value_counts()

0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

### Замена типа данных

In [14]:
clients['total_income'] = clients['total_income'].astype(int)

In [15]:
clients['days_employed'] =clients['days_employed'].astype(int)

In [16]:
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [17]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Для удобства меняем тип данных в days_employed и total_income на целочисленные с помощью функции astype(). Проверяем.

### Обработка дубликатов

In [18]:
clients = clients.drop_duplicates().reset_index(drop=True)

In [19]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 12 columns):
children            21471 non-null int64
days_employed       21471 non-null int64
dob_years           21471 non-null int64
education           21471 non-null object
education_id        21471 non-null int64
family_status       21471 non-null object
family_status_id    21471 non-null int64
gender              21471 non-null object
income_type         21471 non-null object
debt                21471 non-null int64
total_income        21471 non-null int64
purpose             21471 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

 Так как кредиты берут конкретные люди, дубликаты сразу удаляем при в полном совпадении в строках. Провяряем вызовом info количество записей: 21471 вместо изначальных 21525. Дубликаты появились, возможно, по причине заполнения более одной заявки в один момент или технической ошибки.

### Лемматизация

In [20]:
clients['purpose'].value_counts() 

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [21]:
from pymystem3 import Mystem
m = Mystem()
def purpose_lemmas(text):
    lemmas = m.lemmatize(text)
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    else:
        return 'недвижимость'


In [22]:
clients['purpose_group'] = clients['purpose'].apply(purpose_lemmas)

In [23]:
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


### Вывод

посмотрели количество уникальных значений целей получения кредитов, из нее сделали выводы, что всего есть 4 группы целей:
- свадьба
- недвижимость(в эту группу мы отнесли и цели, которые описаны с помощью слов "жилье", "ремонт")
- образование
- автомобиль. 

Написали функцию для определения группы цели для этого используем лемматизацию. 
К таблице добавили название группы целей, проверили.

### Категоризация данных

In [24]:
clients.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21471.000000,21471.000000,21471.000000,21471.000000,21471.000000,21471.000000,2.147100e+04
mean,0.470961,60523.894416,43.279074,0.817195,0.973685,0.081086,1.674218e+05
std,0.750893,133393.029130,12.574291,0.548508,1.421082,0.272974,9.775563e+04
min,0.000000,24.000000,0.000000,0.000000,0.000000,0.000000,2.066700e+04
25%,0.000000,1024.000000,33.000000,1.000000,0.000000,0.000000,1.076545e+05
50%,0.000000,2194.000000,42.000000,1.000000,0.000000,0.000000,1.561050e+05
75%,1.000000,4794.000000,53.000000,1.000000,1.000000,0.000000,1.957515e+05
max,5.000000,401755.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [25]:
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


In [26]:
def total_income_level(income):
    if income < 107655:
        return 'низкий'
    if 107655 <= income < 156105:
        return 'ниже среднего'
    if 156105 <= income <= 195752:
        return 'выше среднего'
    else:
        return 'высокий'

In [27]:
clients['total_income_level'] = clients['total_income'].apply(total_income_level)

In [28]:
clients.head(30)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,total_income_level
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,ниже среднего
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,ниже среднего
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,выше среднего
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,высокий
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,высокий
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,ниже среднего
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,низкий
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,ниже среднего


### Вывод

С помощью функции discribe находим процентили по total_income.

Пишем функцию total_income_level, которая разделит клиентов по уровню дохода на 4 части:
 - низкий
 - ниже среднего
 - выше среднего
 - высокий.
 
Добавляем столбец в таблицу с соответствуйщими значениями.
 

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [29]:
total_debt = clients['debt'].count() # считаем общее число людей 
print(total_debt)

21471


In [30]:
children_grouping = clients.groupby('children')['debt'].sum() # суммируем количество долгов у людей, сгруппированных по семейному положению
print(children_grouping)

children
0    1072
1     444
2     194
3      27
4       4
5       0
Name: debt, dtype: int64


In [31]:
for i in range(len(children_grouping)):  # определяем доли людей, сгруппированных по количеству детей, не вернувших кредит вовремя
        debt_part_children = children_grouping[i] / total_debt 
        print('{:.0%}'.format(debt_part_children))

5%
2%
1%
0%
0%
0%


In [32]:
children_pivot = clients.pivot_table (index = ['children'], values = 'debt', aggfunc = 'sum') # делаем тоже самое, но со сводной таблицей
print(children_pivot)

          debt
children      
0         1072
1          444
2          194
3           27
4            4
5            0


In [33]:
children_pivot['debt_part'] = children_pivot['debt'] / total_debt # добавляем столбец с долей
print(children_pivot)

          debt  debt_part
children                 
0         1072   0.049928
1          444   0.020679
2          194   0.009035
3           27   0.001258
4            4   0.000186
5            0   0.000000


### Вывод

Люди, не имеющие детей задерживали кредит в 5 % случаев.
Имеющие 1 ребенка - в 2%, 2 детей - еще в два раза реже (1%).
Те, у кого трое и более детей, почти не задерживали кредит.


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [34]:
family_status_id_grouping = clients.groupby('family_status_id')['debt'].sum() # суммируем количество долгов у людей, сгруппированных по семейному положению
print(family_status_id_grouping)

family_status_id
0    931
1    388
2     63
3     85
4    274
Name: debt, dtype: int64


In [35]:
for i in range(len(family_status_id_grouping)):  # определяем доли людей, сгруппированных по семейному положению, не вернувших кредит вовремя
        debt_part_family_status_id = family_status_id_grouping[i] / total_debt 
        print('{:.0%}'.format(debt_part_family_status_id))

4%
2%
0%
0%
1%


### Вывод

 Получили следующие показатели:
самое большое число возвратов не вовремя (4%) - у женатых/замужних людей, в два раза меньше у людей в гражданском браке (2%), чуть более надежные - незамужние/неженатые (1%).
Самые надежные кредитополучатели - вдовцы (0%) и люди в разводе(0%).

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [36]:
total_income_level_grouping = clients.groupby('total_income_level')['debt'].sum() # суммируем количество долгов у людей, сгруппированных по семейному положению
print(total_income_level_grouping)

total_income_level
высокий          383
выше среднего    462
ниже среднего    469
низкий           427
Name: debt, dtype: int64


In [37]:
for i in range(len(total_income_level_grouping)):  # определяем доли людей, сгруппированных по уровню дохода, не вернувших кредит вовремя
        debt_part_family_total_income_level = total_income_level_grouping[i] / total_debt 
        print('{:.0%}'.format(debt_part_family_total_income_level))

2%
2%
2%
2%


### Вывод

Процент невозврата вовремя у людей с разными уровнями дохода (группы - низкий, ниже среднего, выше среднего, высокий) одинаковый - 2%, соответственно,  зависимости от дохода нет.

- Как разные цели кредита влияют на его возврат в срок?

In [38]:
purpose_group_grouping = clients.groupby('purpose_group')['debt'].sum() # суммируем количество долгов у людей, сгруппированных по уровню дохода
print(purpose_group_grouping)

purpose_group
автомобиль      403
недвижимость    782
образование     370
свадьба         186
Name: debt, dtype: int64


In [39]:
for i in range(len(purpose_group_grouping)):  # определяем доли людей, сгруппированных по уровню дохода, не вернувших кредит вовремя
        debt_part_purpose_group = purpose_group_grouping[i] / total_debt 
        print('{:.0%}'.format(debt_part_purpose_group))

2%
4%
2%
1%


### Вывод

Хуже всего возвращаются кредиты на недвижимость(4%),  в два раза лучше - на автомобиль и образование.
Лучше всего возвращали кредиты на свадьбу (только 1% задолженностей).

### Шаг 4. Общий вывод

Мы изучили влияние различных факторов задержку возврата кредитов и установили следующие вероятности:
   - Зависимость от наличия детей
       Люди, не имеющие детей задерживали кредит в 5% случаев.
       Имеющие 1 ребенка - в 2%, 2 детей - еще в два раза реже (1%).
       Те, у кого трое и более детей, не задерживали кредит (0%).
   - Зависимость от семейного положения
       Самое большое число возвратов не вовремя (4%) - у женатых/замужних людей,
       в два раза меньше у людей в гражданском браке (2%),
       чуть более надежные - незамужние/неженатые (1%).
       Самые надежные кредитополучатели - вдовцы (0%) и люди в разводе(0%).
       - Зависимости от уровня дохода нет:
       люди с разным доходом (низкий, ниже среднего, выше среднего, высокий) задерживали выплату кредита в 2% случаев.
   - Зависимость от целей
       Хуже всего возвращаются кредиты на недвижимость(4%)
       почти в два раза лучше(2%) - на автомобиль и образование.
       Лучше всего возвращали кредиты на свадьбу (только 1% задолженностей).
       
Таким образом, самые высокие риски для клиентов невозврата кредита в срок у людей:
  - не имеющие детей
  - женатые/замужние
  - с кредитом на недвижимость
    
Лучшие кандидаты на кредит:
  - с двумя и более детьми 
  - овдовевшие или в разводе
  - с кредитом на свадьбу




